# Project: Monitoring NDVI Changes and Segmenting Lost Vegetation

## 1. Introduction
**Objective**: Monitor the changes in NDVI over the years and segment areas of lost vegetation to classify what they have been converted to using the EuroSAT dataset.

**Tools**: Python, Satellite Imagery, NDVI Calculation, EuroSAT Dataset, Machine Learning Models

## 2. Data Collection and Preparation
### Step 1: Collect Satellite Images
- **Sources**: Landsat, Sentinel-2
- **Period**: Define the years for monitoring (e.g., 2000-2020)
- **Regions**: Define the geographic regions of interest

### Step 2: Preprocess Satellite Images
- **Cloud Masking**: Remove clouds and shadows
- **Radiometric Calibration**: Correct sensor noise and inconsistencies

### Step 3: Calculate NDVI
- **Formula**: NDVI = (NIR - RED) / (NIR + RED)
- **Implementation**: Use libraries like `rasterio` and `numpy`

### Step 4: Load EuroSAT Dataset
- **Download**: Obtain the EuroSAT dataset
- **Categories**: Urban, Agriculture, Forest, etc.

## 3. NDVI Analysis
### Step 1: Compute Yearly NDVI Maps
- **Mean NDVI**: Calculate average NDVI for each year
- **Seasonal NDVI**: If needed, calculate for different seasons

Table. Quote: **Holben, B.N. (1986). Characteristics of Maximum- Value Composite Images from Temporal AVHRR Data. International Journal of Remote Sensing, 7(11), 1417-1434.** 
| NDVI Range    | Class                |
|----------------|---------------------|
| <0             | water               |
| 0.03 – 0       | bare soil           |
| 0.03 – 0.3     | sparse vegetation   |
| 0.3 – 0.5      | moderate vegetation |
| >0.5           | dense vegetation    |

### Step 2: Identify Changes in NDVI
- **Trend Analysis**: Use statistical methods to identify significant changes over the years
- **Change Detection**: Highlight areas with significant NDVI reduction

## 4. Segmentation of Lost Vegetation
### Step 1: Identify Lost Vegetation Tiles
- **Thresholding**: Define NDVI thresholds to identify loss
- **Mask Creation**: Create masks for areas of NDVI reduction

### Step 2: Extract Lost Vegetation Tiles
- **Coordinate Extraction**: Get coordinates of identified tiles
- **Tile Extraction**: Extract these regions from satellite images

## 5. Classification Using EuroSAT
### Step 1: Data Preparation
- **Tile Labelling**: Label extracted tiles according to EuroSAT categories
- **Data Augmentation**: If necessary, augment data to improve model performance

### Step 2: Model Selection and Training
- **Model**: Choose a suitable classification model (e.g., CNN)
- **Training**: Train the model using EuroSAT dataset
- **Validation**: Validate model performance on a separate validation set

### Step 3: Classification of Lost Vegetation Tiles
- **Prediction**: Use the trained model to classify lost vegetation tiles
- **Post-processing**: Aggregate and interpret classification results

## 6. Results and Analysis
### Step 1: Visualization
- **NDVI Changes**: Visualize NDVI changes over the years using plots or maps
- **Classified Tiles**: Map out classified lost vegetation tiles

### Step 2: Interpretation
- **Land Use Changes**: Interpret what the lost vegetation has been converted to
- **Environmental Impact**: Discuss potential environmental impacts

## 7. Conclusion
- **Summary**: Summarize key findings
- **Future Work**: Suggest possible extensions or improvements

## 8. References and Documentation
- **References**: List of scholarly articles, datasets, and tools used
- **Documentation**: Detailed code comments and project documentation


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp
from tqdm.auto import tqdm
import pandas as pd
import os

import geemap
import requests

2024-06-24 13:10:43.585499: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 13:10:44.590960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Initialize Earth Engine API

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='cementification')

In [ ]:
imageCollectionInfoUrl = 'https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/COPERNICUS_S2_SR_HARMONIZED.json'
r = requests.get(imageCollectionInfoUrl)
imageCollInfo = r.json()
print(imageCollInfo)

In [ ]:
bands = []

for key, value in imageCollInfo.items():
    if key == 'summaries':
        for key2, value2 in value.items():
            if key2 == 'eo:bands':
                for band in value2:
                    print(f'{band["name"]}: {band}')
                    bands.append(band['name'])

In [ ]:
city = 'naples'

geometry = ee.FeatureCollection(f'projects/cementification/assets/{city}_10km_buffer').geometry()

# Load the Sentinel-2 image collection
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2017-03-28', '2024-06-01') \
                .filter(ee.Filter.calendarRange(3, 9, 'month')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .select(bands)

print(collection.size().getInfo())

In [ ]:
# Select only these bands to reduce size of exported images
bands = ['B2', 'B3', 'B4', 'B8']

In [ ]:
if city == 'milan':
    geometry = ee.Geometry.Polygon(
        [[[8.967864766059032, 45.73943107755818],
          [8.967864766059032, 45.41260677458684],
          [9.434783711371532, 45.41260677458684],
          [9.434783711371532, 45.73943107755818]]])
elif city == 'naples':
    geometry = ee.Geometry.Polygon(
        [[[13.829280147955982, 41.1031243453332],
          [13.829280147955982, 40.51481111232291],
          [14.587336788580982, 40.51481111232291],
          [14.587336788580982, 41.1031243453332]]])

In [ ]:
# Function to compute NDVI
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI').multiply(10000).toInt16()
    return image.addBands(ndvi)

# Function to filter the collection by season and compute median NDVI
def median_ndvi_for_season(year, season_start_month):
    start_date = ee.Date.fromYMD(year, season_start_month, 1)
    end_date = start_date.advance(3, 'month')
    season_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .select(bands) \
                .map(compute_ndvi) \
                .select(['B2', 'B3', 'B4', 'NDVI'])
    
    # Determine season string based on month
    is_spring = ee.Number(season_start_month).eq(3)
    season_string = ee.String(ee.Algorithms.If(is_spring, 'spring', 'summer'))
    return season_collection.median().clip(geometry).set('year', year).set('season', season_string)

# Define seasons
seasons = ee.List([3, 6])

# Generate a list of years
years = ee.List.sequence(2017, 2023)

# Compute the median NDVI for each year and season
def map_years(year):
    def map_seasons(season_start_month):
        return median_ndvi_for_season(year, season_start_month)
    return seasons.map(map_seasons)

year_season_combinations = years.map(map_years).flatten()

# Convert the list of images to an image collection
seasonal_ndvi_collection = ee.ImageCollection(year_season_combinations)

# Print the size of the seasonal NDVI collection
print(f'Adding {seasonal_ndvi_collection.size().getInfo()} seasonal layers')

# Create a map
Map = geemap.Map()

# Define visualization parameters for NDVI
vis_param_ndvi = {
    'min': -1, 
    'max': 1, 
    'bands': ['NDVI'],
    'palette': ['blue', 'white', 'green']}

# Create a progress bar
total_iterations = len(seasons.getInfo()) * len(years.getInfo())
progress_bar = tqdm(total=total_iterations)

# Export each seasonal median NDVI
for year in range(2017, 2024): # From 2017 to 2024
    for season in ['spring', 'summer']:     # Spring and Summer
        season_ndvi = seasonal_ndvi_collection.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('season', season)).first()

        if season_ndvi is None:
            print(f"Skipping {city} {year} {season}")
            progress_bar.update()
            continue
        
        # Ensure consistent band data types
        season_ndvi = season_ndvi.toInt16()
        
        # # Export the image to Google Drive
        # task = ee.batch.Export.image.toDrive(
        #     image=season_ndvi,
        #     description=f"{city}_{year}_{season}",
        #     folder="Environment_exam_seasonal",
        #     fileNamePrefix=f"{city}_{year}_{season}",
        #     scale=10,
        #     region=geometry.getInfo()['coordinates'],
        #     fileFormat='GeoTIFF',
        #     skipEmptyTiles=True
        # )
        # task.start()
        
        progress_bar.update()  # Update the progress bar

# Close the progress bar
progress_bar.close()


In [ ]:
# Initialize an empty list to store the data
data = []

for img in os.listdir('images'):
    city, year, season_ext = img.split('_')
    season, _ = season_ext.split('.')
    # Append a dictionary with the data to the list
    data.append({'city': city, 'year': year, 'season': season, 'path': f'images/{img}'})

# Create a DataFrame from the list
img_df = pd.DataFrame(data, columns=['city', 'year', 'season', 'path', 'water', 'roads_or_urban_areas', 'vegetation'])
img_df

In [ ]:
# Define the NDVI thresholds
ndvi_thresholds = {
    'water': (-10000, 0),
    'roads_or_urban_areas': (0, 2000),
    'vegetation': (2000, 10000)
}

# Define a function to classify NDVI values
def compute_percentages_by_threshold(ndvi_img, thresholds):
    # Initialize an empty dictionary to store the percentages
    percentages = {}

    # # Iterate over the thresholds
    # for key, value in thresholds.items():
    #     # Compute the percentage of pixels within the threshold
    #     percentage = ndvi_img.gt(value[0]).And(ndvi_img.lte(value[1])).reduceRegion(
    #         reducer=ee.Reducer.mean(),
    #         geometry=geometry,
    #         scale=10,
    #         maxPixels=1e9
    #     ).values().get(0)

    #     # Add the percentage to the dictionary
    #     percentages[key] = percentage

    # Assuming ndvi_img is a numpy.ndarray and thresholds is a dictionary
    # with keys as labels and values as a tuple of (lower_bound, upper_bound)
    percentages = {}
    for key, value in thresholds.items():
        # Create a boolean mask where values within the threshold are True
        mask = (ndvi_img > value[0]) & (ndvi_img <= value[1])
        # Compute the percentage of True values in the mask
        percentage = np.mean(mask) * 100  # Convert fraction to percentage
        percentages[key] = percentage
    
    return percentages

In [ ]:
from sklearn.cluster import KMeans
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

test_img = img_df['path'][20]
print('img: ', test_img)
with rasterio.open(test_img) as src:
    ndvi_img = src.read(4)  # Read the NDVI band
    plt.hist(ndvi_img.flatten(), bins=250, color='c', edgecolor='k', alpha=0.7)
    plt.show()
    # Reshape the image to a 2D array
    image_2d = ndvi_img.reshape(-1, 1)

    # Apply K-means clustering with 3 clusters
    kmeans = KMeans(n_clusters=4, random_state=0)
    kmeans.fit(image_2d)

    # Get the cluster labels
    labels = kmeans.labels_

    # Reshape the labels back to the original image shape
    clustered_image = labels.reshape(ndvi_img.shape)

    # Display the clustered image
    # plt.imshow(clustered_image, cmap='viridis')
    # plt.colorbar()
    # plt.show()
    # Display the clustered image with a discrete color map

    # Your list of colors
    cmap = ['blue', 'grey', 'green']

    # Create a colormap from the list
    custom_cmap = ListedColormap(cmap)

    plt.imshow(clustered_image, cmap=custom_cmap)
    plt.colorbar()
    plt.show()

In [ ]:
import rasterio
import matplotlib.pyplot as plt

for img_path in tqdm(img_df['path']):
    with rasterio.open(img_path) as src:
        ndvi_band = src.read(4)
        percentages = compute_percentages_by_threshold(ndvi_band, ndvi_thresholds)
        img_df.loc[img_df['path'] == img_path, 'water'] = percentages['water']
        img_df.loc[img_df['path'] == img_path, 'roads_or_urban_areas'] = percentages['roads_or_urban_areas']
        img_df.loc[img_df['path'] == img_path, 'vegetation'] = percentages['vegetation']

print(img_df)

In [ ]:
sorted_df = img_df.sort_values(by=['city', 'year', 'season'], inplace=False)
sorted_df

In [ ]:
import rasterio

import matplotlib.pyplot as plt

# Normalize and scale each channel
def normalize_and_scale_uint8(channel):
    # Normalize to 0...1
    min_val = channel.min()
    max_val = channel.max()
    channel_normalized = (channel - min_val) / (max_val - min_val)
    # Scale to 0...255 and convert to uint8
    channel_scaled = np.clip(channel_normalized * 255, 0, 255).astype(np.uint8)
    return channel_scaled

# Open the image
with rasterio.open('images/naples_2022_summer.tif') as src:
    # Get the band names
    band_names = src.descriptions
    print(band_names)
    # Read the NDVI band
    ndvi_band = src.read(4)
    print(f'min ndvi value: {ndvi_band.min()} max ndvi value: {ndvi_band.max()}')

    red_band = src.read(3)
    green_band = src.read(2)
    blue_band = src.read(1)

    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)
    plt.imshow(rgb_image * 1e-4)
    plt.title('RGB Image scaled 1e-4')
    plt.show()

    # Normalize and scale each channel
    red_scaled = normalize_and_scale_uint8(red_band)
    green_scaled = normalize_and_scale_uint8(green_band)
    blue_scaled = normalize_and_scale_uint8(blue_band)

    # Stack the channels
    rgb_image = np.stack((red_scaled, green_scaled, blue_scaled), axis=-1)
    plt.imshow(rgb_image)
    plt.title('RGB Image')
    plt.show()

# Plot the NDVI band
plt.imshow(ndvi_band, cmap='RdYlGn')
plt.colorbar(label='NDVI')
plt.title('NDVI Band')
plt.show()

In [ ]:
run scripts/dispms -f images/naples_2017_spring.tif -e 5 -p [3,2,1] -d [0,0,5199,3642]

In [ ]:
run scripts/dispms -f images/naples_2017_summer.tif -e 4 -p [2,4,3] -d [0,0,5199,3642]

# Using Landsat 7 images (from 1999 to 2022, 30m resolution)

In [ ]:
# Function to compute NDVI
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI').multiply(10000).toInt16()
    return image.addBands(ndvi)

# Function to filter the collection by season and compute median NDVI
def median_ndvi_for_year(year):
    start_date = ee.Date.fromYMD(year, 3, 1)
    end_date = start_date.advance(6, 'month')
    year_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
                .filterBounds(geometry) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4']) \
                .map(compute_ndvi) \
                .select(['SR_B1', 'SR_B2', 'SR_B3', 'NDVI'])
    return year_collection.median().clip(geometry).set('year', year)

# Generate a list of years
years = ee.List.sequence(1999, 2023)

year_ndvi = years.map(median_ndvi_for_year).flatten()

# Convert the list of images to an image collection
years_ndvi_collection = ee.ImageCollection(year_ndvi)

# Print the size of the seasonal NDVI collection
print(f'Adding {years_ndvi_collection.size().getInfo()} annual layers')

# Create a map
Map = geemap.Map()

# Define visualization parameters for NDVI
vis_param_ndvi = {
    'min': -1, 
    'max': 1, 
    'bands': ['NDVI'],
    'palette': ['blue', 'white', 'green']}

# Create a progress bar
total_iterations = len(years.getInfo())
progress_bar = tqdm(total=total_iterations)

# Export each annual median NDVI
for year in range(1999, 2024): # From 1999 to 2023
    year_ndvi = years_ndvi_collection.filter(ee.Filter.eq('year', year)).first()
    
    # Ensure consistent band data types
    year_ndvi = year_ndvi.toInt16()
    
    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=year_ndvi,
        description=f"{city}_{year}",
        folder="Environment_exam_annual",
        fileNamePrefix=f"{city}_{year}",
        scale=30,
        region=geometry.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        skipEmptyTiles=True
    )
    task.start()
    
    progress_bar.update()  # Update the progress bar

# Close the progress bar
progress_bar.close()